In [1]:
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper
import dill
import numpy as np
from qiskit_algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit.circuit.library import EfficientSU2

In [2]:
import qiskit_nature
qiskit_nature.__version__

'0.7.2'

In [3]:
BL = 0.6

In [4]:
# Parameters
BL = 0.9


### 1 Construct the Hamiltonian

Helper function to construct the Hamiltonian represented as a Pauli decomposition stored in `qubit_op`, while also returning the `nuclear_repulsion_energy` and `core_electron_energy`.

In [5]:
def get_qubit_op(BL):
    molecule = MoleculeInfo(
        symbols=["H", "H"],
        coords=([0.0, 0.0, 0.0], [BL, 0.0, 0.0]),
        multiplicity=1,
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    properties = driver.run()

    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(properties)

    # Get the fermionic Hamiltonian (first element)
    fermionic_op, _ = problem.second_q_ops()

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    # Use Jordan-Wigner (no tapering)
    mapper = JordanWignerMapper()
    qubit_op = mapper.map(fermionic_op)
    # converter = QubitConverter(mapper=mapper)
    # qubit_op = converter.convert(fermionic_op, num_particles=num_particles)

    core_electron_energy = problem.hamiltonian.constants["FreezeCoreTransformer"]
    nuclear_repulsion_energy = problem.nuclear_repulsion_energy

    return (
        qubit_op,
        num_particles,
        num_spatial_orbitals,
        problem,
        mapper,
        nuclear_repulsion_energy,
        core_electron_energy,
    )


In [6]:
(qubit_op, num_particles, num_spatial_orbitals, problem, mapper,nuclear_repulsion_energy, core_electron_energy) = get_qubit_op(BL)

In [7]:
qubit_op

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.8470288 +0.j,  0.14907479+0.j, -0.16071249+0.j,  0.11162723+0.j,
  0.14907479+0.j,  0.16113816+0.j, -0.16071249+0.j,  0.15927016+0.j,
  0.04764292+0.j,  0.04764292+0.j,  0.04764292+0.j,  0.04764292+0.j,
  0.15927016+0.j,  0.16737126+0.j,  0.11162723+0.j])

In [8]:
# Extract terms and coefficients
pauli_list = list(zip(qubit_op.paulis, qubit_op.coeffs))

# Sort terms by absolute coefficient magnitude (largest first)
sorted_pauli_list = sorted(pauli_list, key=lambda x: abs(x[1]), reverse=True)

# Print sorted terms
# for pauli, coef in sorted_pauli_list:
#     print(f"Pauli: {pauli}, Coefficient: {coef}")

# Save the list
with open("sorted_pauli_list.pkl", "wb") as f:
    dill.dump(sorted_pauli_list, f)

In [9]:
dd = np.real(min(np.linalg.eigvals(qubit_op.to_matrix())))

### 2 Perform VQE

Define a class to perform VQE.

In [10]:
# Define the custom VQE
class CustomVQE:
    def __init__(self, estimator, ansatz, optimizer, qubit_op, dd, NREplusCEE, custom_function, initial_point=None):
        self.estimator = estimator
        self.ansatz = ansatz
        self.optimizer = optimizer
        self.qubit_op = qubit_op
        self.NREplusCEE = NREplusCEE
        self.dd = dd
        self.custom_function = custom_function
        self.initial_point = initial_point  or [0] * ansatz.num_parameters

    def compute_minimum_eigenvalue(self):
        # Define the function to minimize
        def objective_function(params):
            # Evaluate the expectation value of the qubit operator
            expectation = self.estimator.run([self.ansatz], [self.qubit_op], [params]).result().values[0]
            # Evaluate the custom objective function
            return self.custom_function(expectation, params)

        # Use the `minimize` method for optimization
        opt_result = self.optimizer.minimize(
            fun=objective_function,
            x0=self.initial_point
        )

        result = {
            'x': [float(value) for value in opt_result.x],
            'fun': opt_result.fun,
            'dd': self.dd,
            'NREplusCEE': self.NREplusCEE
        }

        
        return result


In [11]:
# l=12 and maxiter = 200 works for LiH and HF

In [12]:
optimizer = SLSQP(maxiter=300)
noiseless_estimator = Estimator(approximation=True)


# Define ansatz
var_form = EfficientSU2(qubit_op.num_qubits, reps=12, entanglement='full', skip_unentangled_qubits=False, parameter_prefix='a')


# Example Custom Objective Function
def custom_objective_function(expectation_value, parameters):
    # penalty_term = 0.1 * sum(param**2 for param in parameters)  # Example regularization
    return expectation_value 


# Create an instance of CustomVQE
custom_vqe = CustomVQE(
    estimator=noiseless_estimator,
    ansatz=var_form,
    optimizer=optimizer,
    qubit_op=qubit_op,
    dd = dd,
    NREplusCEE = nuclear_repulsion_energy + core_electron_energy,
    custom_function=custom_objective_function,
    initial_point = list(np.random.uniform(-np.pi, np.pi, var_form.num_parameters))
)


In [13]:
vqe_result = custom_vqe.compute_minimum_eigenvalue()

In [14]:
vqe_result

{'x': [-2.6075164649215394,
  -3.0916065122303746,
  -2.2647317687183777,
  1.3475244864752227,
  1.2877796697275532,
  -0.16381018861951405,
  2.8374199957459023,
  -0.19952276946017597,
  -0.5051760680118071,
  -2.491094675675747,
  -0.7035918080416497,
  -0.009920341174781312,
  -2.8479485055353173,
  -1.5633074047682187,
  -0.6493819086339467,
  1.3764602236454497,
  -1.8328096327558845,
  0.3772067870323669,
  -0.8098816106400973,
  -0.2268622259733506,
  -1.0409882487470752,
  -1.9370655298776724,
  -1.0837153702433018,
  -1.1148834955570448,
  -1.3671968542517772,
  -1.082026303855216,
  -1.2753785468121102,
  2.3031949284795785,
  1.4617034488776057,
  -0.5624505084046397,
  -1.5201983377238149,
  -2.711032404232417,
  1.4268455330838357,
  3.2642116278789124,
  1.826129295641071,
  -2.2069795141243334,
  -2.109876073248932,
  -0.7403465632975633,
  1.3076246063803478,
  2.0666464229274877,
  2.4343005297364946,
  -1.9266393405093223,
  -2.975594828484925,
  0.7640714466188243,

In [15]:
len(vqe_result['x'])

104

In [16]:
print(vqe_result['fun'] - vqe_result['dd'])

6.409108517235751e-07


In [17]:
with open("vqe_result.pkl", "wb") as f:
    dill.dump(vqe_result, f)